# Most Frustrated Spins from Different Metrics

In [ ]:
using QAOA
using HDF5, Printf

PATH = "/home/ubuntu/Archives/"
PLOT_PATH = "/home/ubuntu/Archives/plots/SK_model/paper/";

In [ ]:
using Revise, SpinFluctuations

In [ ]:
N = 9
# N = 11
# N = 13
# N = 15
# N = 17
# N = 19

patterns_dict = Dict(
    9  => r"random_SK_instance_N_9_seed_(\d+)\.h5",
    11 => r"random_SK_instance_N_11_seed_(\d+)\.h5",
    13 => r"random_SK_instance_N_13_seed_(\d+)\.h5",
    15 => r"random_SK_instance_N_15_seed_(\d+)\.h5",
    17 => r"random_SK_instance_N_17_seed_(\d+)\.h5",
    19 => r"random_SK_instance_N_19_seed_(\d+)\.h5"
);

In [ ]:
subdir = "small_gaps"
# subdir = "large_gaps"
folder_name = PATH * @sprintf("data/SK_model/N_%i/%s/", N, subdir)
instance_names = readdir(folder_name)
filter!(x -> !occursin("results", x), instance_names)
filter!(x -> !occursin("undecided", x), instance_names)
filter!(x -> !occursin("frustrated", x), instance_names);

In [ ]:
# # testing
# instance_names = instance_names[1:10]

### From area under magnetization

In [ ]:
# T_final = 32000.
T_final = 32768.
tol = 1e-6

npts = 2048
coarse_times = range(0, 1, npts + 1);

In [ ]:
all_most_frustrated_spins = Dict()

for instance_name in instance_names
    seed = match(patterns_dict[N], instance_name)[1]  
    print(seed, "\t")  
    
    # Mean-field trajectories
    sol_t = h5read(folder_name * "results_" * instance_name, @sprintf("mean_field_T_final_%.0f_tol_1e%.0f/times", T_final, log10(tol)))
    sol_u = h5read(folder_name * "results_" * instance_name, @sprintf("mean_field_T_final_%.0f_tol_1e%.0f/trajectories", T_final, log10(tol)))
    nz_vals = n_vals("z", sol_u)
    nxy_coarse = zeros(N-1)
    nz_coarse = n_coarse(nz_vals, sol_t, coarse_times)

    # mean-field
    J_mat = h5read(folder_name * instance_name, "J")
    mf_problem = Problem(0, J_mat)

    S_vals = [transpose(reduce(hcat, [nxy_coarse, nxy_coarse, nz_coarse[:, k]])) |> Matrix for k in 1:npts+1]
    magnetizations = reduce(hcat, map(S -> magnetization(S, mf_problem.local_fields, mf_problem.couplings), S_vals));  

    # Get most frustrated spins from area under magnetization
    areas = Dict()
    dts = [(x[2] - x[1]) / T_final for x in zip(coarse_times[1:end-1], coarse_times[2:end])]
    for spin_idx in 1:N-1
        areas[spin_idx] = sum(dts .* magnetizations[spin_idx, 2:end]) |> abs
    end
    all_most_frustrated_spins[seed] = [k for (k, v) in sort(areas |> collect, by=x->x[2])]
end

In [ ]:
all_most_frustrated_spins

In [ ]:
# h5write(folder_name * @sprintf("most_frustrated_spins_N_%i.h5", N), 
# @sprintf("T_final_%.0f_tol_1e%.0f/seeds", 32768., log10(1e-6)), [k |> string for (k, v) in all_most_frustrated_spins])

# h5write(folder_name * @sprintf("most_frustrated_spins_N_%i.h5", N), 
# @sprintf("T_final_%.0f_tol_1e%.0f/spin_idxs", 32768., log10(1e-6)), reduce(hcat, [v for (k, v) in all_most_frustrated_spins]) |> transpose |> Matrix)

### From area under $n^z$ components

In [ ]:
most_undecided_spins = Dict()

for instance_name in instance_names
    seed = match(patterns_dict[N], instance_name)[1]    
    
    # Mean-field trajectories
    sol_t = h5read(folder_name * "results_" * instance_name, @sprintf("mean_field_T_final_%.0f_tol_1e%.0f/times", 32768., log10(1e-6)))
    sol_u = h5read(folder_name * "results_" * instance_name, @sprintf("mean_field_T_final_%.0f_tol_1e%.0f/trajectories", 32768., log10(1e-6)))
    nzs = reduce(hcat, [sol_u[k, 3, :] for k in 1:size(sol_u)[1]])

    # Get "most undecided spin" from area under z components
    areas = Dict()
    dts = [(x[2] - x[1]) / T_final for x in zip(sol_t[1:end-1], sol_t[2:end])]
    for spin_idx in 1:N-1
        areas[spin_idx] = sum(dts .* nzs[spin_idx, 2:end]) |> abs
    end
    undecided_spins = [k for (k, v) in sort(areas |> collect, by=x->x[2])]
    most_undecided_spin = undecided_spins[1]
    most_undecided_spins[seed] = most_undecided_spin
end

In [ ]:
most_undecided_spins

In [ ]:
# h5write(folder_name * @sprintf("most_undecided_spins_N_%i.h5", N), 
# @sprintf("T_final_%.0f_tol_1e%.0f/seeds", 32768., log10(1e-6)), [k for (k, v) in most_undecided_spins])

# h5write(folder_name * @sprintf("most_undecided_spins_N_%i.h5", N), 
# @sprintf("T_final_%.0f_tol_1e%.0f/spin_idxs", 32768., log10(1e-6)), [v for (k, v) in most_undecided_spins])

### From energy change upon spin flip

In [ ]:
all_most_frustrated_spins = Dict()

for instance_name in instance_names
    seed = match(patterns_dict[N], instance_name)[1]    
    
    # Mean-field trajectories
    sol_t = h5read(folder_name * "results_" * instance_name, @sprintf("mean_field_T_final_%.0f_tol_1e%.0f/times", 32768., log10(1e-6)))
    sol_u = h5read(folder_name * "results_" * instance_name, @sprintf("mean_field_T_final_%.0f_tol_1e%.0f/trajectories", 32768., log10(1e-6)))
    nzs = reduce(hcat, [sol_u[k, 3, :] for k in 1:size(sol_u)[1]])
    
    # mean-field solution and energy
    J_mat = h5read(folder_name * instance_name, "J")
    mf_problem = Problem(0, J_mat)
    
    mf_sol = h5read(folder_name * "results_" * instance_name, @sprintf("mean_field_sol_T_final_%.0f_tol_1e%.0f", T_final, log10(tol)))
    sigma_star = sign.(mf_sol)
    h = mf_problem.local_fields
    J = mf_problem.couplings
    E_star = sum([-h[l] * sigma_star[l] for l in 1:N-1]) + sum([-J[i, j] * sigma_star[i] * sigma_star[j] for i in 1:N-1 for j in (i+1):N-1])
    
    # Get "most frustrated spin" from spin flip
    most_frustrated_spins = [(0, 0.) for _ in 1:N-1]
    for spin_idx in 1:N-1
        sigma_star_flip = sign.(sol_u[end, 3, :])
        sigma_star_flip[spin_idx] = (-1) * sigma_star_flip[spin_idx]
        E_flip = sum([-h[l] * sigma_star_flip[l] for l in 1:N-1]) + sum([-J[i, j] * sigma_star_flip[i] * sigma_star_flip[j] for i in 1:N-1 for j in (i+1):N-1])
        most_frustrated_spins[spin_idx] = (spin_idx, E_flip)
    end
    all_most_frustrated_spins[seed] = [idx for (idx, E_val) in sort!(most_frustrated_spins, by=x->abs(x[2] - E_star))]
end

In [ ]:
all_most_frustrated_spins

In [ ]:
# h5write(folder_name * @sprintf("most_frustrated_spin_flips_N_%i.h5", N), 
# @sprintf("T_final_%.0f_tol_1e%.0f/seeds", 32768., log10(1e-6)), [k |> string for (k, v) in all_most_frustrated_spins])

# h5write(folder_name * @sprintf("most_frustrated_spin_flips_N_%i.h5", N), 
# @sprintf("T_final_%.0f_tol_1e%.0f/spin_idxs", 32768., log10(1e-6)), reduce(hcat, [v for (k, v) in all_most_frustrated_spins]) |> transpose |> Matrix)